###  En este cuaderno vamos a hacer algo muy similar a lo que hicimos en el cuaderno sobre daltonismo. En ese cuaderno, a una imagen con 2-Dimensiones de color, le agregamos una tercera dimension con pixeles negros. 
<br>
### Ahora, a una imagen con 3-Dimensiones de color - RGB - le vamos a agregar 2 Dimensiones extra con pixeles rojo y azul. Esto nos va a dar información de dónde hay luz Ultravioleta e Infrarojo! 
<br>
### Usaremos una imagen de una galaxia para este ejercicio. 

# Importar módulos

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#http://imageio.readthedocs.io/en/latest/installation.html
import imageio
import os
import pylab
import numpy as np
%pylab inline


# Definir funciones

Esta función genera una película en formato .gif a partir de una serie de imágenes en formato .png

In [ ]:
def makemovie(n, movie_name = 'movie.gif'):
    images = []
    filenames = []
    for i in range(n):
        filenames.append("frame"+str(i)+".png")
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(movie_name, images)

Esta función mapea los pixeles de una imagen al intervalo [0,1] usando una transformación lineal.  

In [ ]:
def linear_map(x):
    """Map data linearly to [0,1] interval."""
    m = np.min(x)
    M = np.max(x)
    return (x-m)/(M-m)

Esta funciona muestra una imagen de manera "bonita"

In [ ]:
def visualizar_imagen(imagen):
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111)
    ax.imshow(imagen, origin="lower", interpolation="gaussian")
    ax.set_axis_bgcolor('k')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Esta función muestra un histograma de los valores en una matriz/imagen

In [ ]:
def histograma_imagen(imagen):
    tamano = 1
    for s in imagen.shape:
        tamano *= s
    img_vec = np.reshape(imagen, tamano)
    fig = plt.figure(figsize=(6,4))
    ax = fig.add_subplot(111)
    hist = plt.hist(img_vec, 50)
    plt.xlabel('Valores pixeles', fontsize = 20)

# Leer los archivos con datos en los 5 canales (RGB y Ultraviolete e Infrarojo): 

Estos datos fueron descargados de este sitio: 

http://skyview.gsfc.nasa.gov/current/cgi/query.pl

Buscamos a la galaxia M51, y tomamos los datos SDSSdr7[g,i,r,u,z]

In [ ]:
# Los archivos que corresponden a R, G, y B se llaman 'i', 'r', y 'g'
R_fits = fits.open('i.fits')
G_fits = fits.open('r.fits')
B_fits = fits.open('g.fits')


#Los archivos .fits con datos UV e IR se llaman u.fits y z.fits. 
# Haz el equivalente con ellos que lo que hicimos con los tres archivos de arriba. 
# asignalos a UV_fits y IR_fits
########


    
########

Los astrónomos trabajan con una archivos que se llaman formato "FITS". Adentro de estos están los datos crudos de las imágenes. En las siguientes lineas, tomamos las imágenes R, G, B, UV y IR

In [ ]:
img_R = R_fits['PRIMARY'].data
img_G = G_fits['PRIMARY'].data
img_B = B_fits['PRIMARY'].data

#Abre las imagenes img_UV y imv_IR
########


########

## Combinamos R, G, y B en una sola imagen

Primero, averiguamos el ancho y alto de la imagen

In [ ]:
ancho_imagen = img_R.shape[0]
altura_imagen = img_R.shape[1]

print('ancho', ancho_imagen, 'pixeles')
print('alto', altura_imagen, 'pixeles')

In [ ]:
# Ahora, generamos una matriz de 3 dimensiones llena de puros ceros. 
img_RGB = np.zeros((ancho_imagen, altura_imagen, 3))

# Ahora, ponemos img_R (el rojo) la posición en img[:,:,0], y img_G (el verde) en img[:,:,1]
img_RGB[:,:,0] = img_R
img_RGB[:,:,1] = img_G

# Pon img_B (el azul) la posición en img_RGB[:,:,2]

########

########

# Cómo se ve la imagen de la galaxia ahorita?

In [ ]:
#Usa la funcion visualizar_imagen para visualizar a img_RGB
################

################

Por qué se ve tan rara? Vamos a ver qué valores tienen los pixeles:

In [ ]:
#Usa la funcion histograma_imagen para ver el histograma de valores de pixeles
##############

###############

#### Vemos que los pixeles no van del 0 al 1, sino que tienen valores mucho más grandes (1000, 2000, etc)

Una forma común en Astronomía de transformar los valores para que vayan del 0 al 1 es la siguiente: sacar el logaritmo base 10 y luego hacer un "mapeo lineal" de 0 a 1

In [ ]:
# 1. sacamos el logaritmo base 10 de los pixeles 
img_RGB_log10 = np.log10(img_RGB)
# 2. lamamos a la función "linear map" 
img_RGB_01 = linear_map(img_RGB_log10)


#### Verificamos los nuevos valores viendo el nuevo histograma: 

In [ ]:
#Grafica el histograma de la nueva imagen, normalizada. 
################

################

Ahora sí van del 0 al 1 los valores de los pixeles!

# Cómo se ve la imagen nueva?

In [ ]:
#Visualiza la nueva imagen.
##################

##################

#### La imagen arriba tiene 3 canales - RGB ... 
<br>
#### Pero lo emocionante es que tenemos información sobre 2 "dimensiones de color" adicionales - el ultravioleta (UV) y el infrarojo (IR) (los datos almacenados en las variables img_UV e img_IR)
<br>
#### Entonces, de manera similar a como le hicimos con el ejercicio para daltónicos, queremos usar *oscilaciones en el tiempo* de pixeles para codificar las 2 dimensiones extras de color. 

# Primero, vamos a transformar de la misma forma que hicimos arriba, los datos de UV e IR a un rango que vaya del 0 al 1.

In [ ]:
# Transforma los datos Ultravioleta (img_UV) e Infrarojo (img_IR) al rango [0,1]. Primero saca
# el logaritmo base 10 de la señal, y luego aplica el mapa lineal (lineal_map)
# Asigna las nuevas imagenes a las variables img_UV_01, img_IR_01

#################


#################

## Veamos el histograma de valores de estos dos canales. Esto nos va a ayudar a decidir qué umbral de intensidad utilizar para seleccionar los pixeles que oscilen en el tiempo. 

In [ ]:
# Muestra el histograma de valores de pixeles del canal UV (transformado al rango 0 - 1)
############# 

#############

In [ ]:
# Muestra el histograma de valores de pixeles del canal IR (transformado al rango 0 - 1)
####### 


#######

# En base a los histogramas de UV e IR, qué valores escogerías para los umbrales de cada uno de estos dos canales? 
<br>
Recuerda que todos los pixeles que tengan un valor por arriba del umbral van a oscilar en el tiempo, mostrando donde hay regiones con alta señal en el canal UV o IR, respectivamente. 

In [ ]:
# Escoge el valor para el umbral del canal UV. Puedes cambiarlo en base a lo que observes
# en el histograma. 
umbral_UV = 0.1

# Escoge el valor para el umbral del canal IR. 
umbral_IR = 0.1

### Hagamos una primera prueba con pixeles en el canal UV que estén por arriba del umbral_UV:

Condición -- Si un pixel en el canal UV está por arriba del umbral_UV, y un número aleatorio entre 0 y 1 es menor a 0.2, pinta ese pixel de color azul. 

In [ ]:
#Has una copia de img_RGB_01, llamala img_prueba_UV
#############


#############

In [ ]:
for i in range(ancho_imagen):  #Este es un doble for loop ...
    for j in range(altura_imagen): # ... que itera sobre los renglones y columnas de la matriz
        # Utiliza un "if statement" para que cada vez se satisfaga la condicion "Si un pixel en 
        # el canal UV está por arriba del umbral_UV, y un número aleatorio entre 0 y 1 es 
        # menor a 0.2, pinta ese pixel de color azul.
        ################################

        
        
        #################################

Cómo se ve esta imagen de prueba?

In [ ]:
#Visualiza la nueva imagen, img_prueba_UV
##################

##################

Qué pasa si cambias el valor del umbral_UV? inténtalo y vuelve a generar la imagen de prueba. 
<br><br>
Algunos valores sugeridos: umbral_UV = 0.05 ó .15, etc. 

### Hagamos ahora una primera prueba con pixeles en el canal IR que estén por arriba del umbral_IR:

Condición -- Si un pixel en el canal IR está por arriba del umbral_IR, y un número aleatorio entre 0 y 1 es menor a 0.2, pinta ese pixel de color rojo. 

In [ ]:
# Haz lo mismo que hicimos arriba, ahora para el canal IR. 
###############

###############

In [ ]:
# Haz lo mismo que hicimos arriba, ahora para el canal IR. 

#################



################

In [ ]:
visualizar_imagen(img_prueba_IR)

Qué pasa si cambias el valor del umbral_IR? inténtalo y vuelve a generar la imagen de prueba. 
<br><br>
Algunos valores sugeridos: umbral_UV = 0.05 ó .15, etc. 

También puedes intentar cambiar el umbral del número aleatorio, para cambiar la densidad de los pixeles. 

### Ahora hagamos una prueba donde los pixeles que esten arriba de los umbrales * de los dos canales * sean de color azul o rojo respectivamente.

Junta las dos condiciones de arriba: 

In [ ]:
img_prueba_IR_UV = img_RGB_01.copy()

In [ ]:
for i in range(ancho_imagen):
    for j in range(altura_imagen):
        if img_IR_01[i,j] > umbral_IR and np.random.random() < 0.2:
            img_prueba_IR_UV[i,j,0] = 1
            img_prueba_IR_UV[i,j,1] = 0
            img_prueba_IR_UV[i,j,2] = 0
        if img_UV_01[i,j] > umbral_UV and np.random.random() < 0.2:
            img_prueba_IR_UV[i,j,0] = 0
            img_prueba_IR_UV[i,j,1] = 0
            img_prueba_IR_UV[i,j,2] = 1

In [ ]:
visualizar_imagen(img_prueba_IR_UV)

# Animación: 
<br>
# Finalmente, vamos a juntar todo lo que hemos construido hasta ahora para hacer una animación de pixeles oscilando en regiones de la imagen donde hay alta intesidad de UV o Infrarojo. 

In [ ]:
# El número de imágenes del cual estará formada la animación. 
num_imagenes = 20

In [ ]:
for n in range(num_imagenes):
    
    # Hacemos una copia de nuestra imagen RGB, a la cual le vamos a cambiar los colores
    # de los pixeles que estén por arriba de los umbrales UV e IR. 
    img_IR_UV = img_RGB_01.copy()
    
    print(n)
    
    # repite lo que hicimos arriba, con las condiciones de UV e IR, pero dentro del 
    # for loop que itera sobre el numero de imagenes. 
    ################################

    
                
    #################################
    visualizar_imagen(img_IR_UV)
    pylab.savefig("frame"+str(n)+".png")


In [ ]:
makemovie(n, 'pelicula_UV_IR.gif')

In [ ]:
![](pelicula_UV_IR.gif)